In [63]:
# Step 1: Load mutation file
mut_df = pd.read_csv("TCGA.STAD.mutations.txt", sep="\t")

# Step 1.5: Keep only SNPs
mut_df = mut_df[mut_df['Variant_Type'] == 'SNP']

# Step 2: Filter for coding mutations of interest
syn = ['Silent']
nonsyn = ['Missense_Mutation', 'Nonsense_Mutation']
mut_df = mut_df[mut_df['Variant_Classification'].isin(syn + nonsyn)]

# Step 3: Remove hypermutator samples (e.g. top 1% by mutation load)
mut_counts = mut_df['Tumor_Sample_Barcode'].value_counts()
threshold = mut_counts.quantile(0.99)
keep_samples = mut_counts[mut_counts <= threshold].index
mut_df = mut_df[mut_df['Tumor_Sample_Barcode'].isin(keep_samples)]

# Step 4: Classify mutations as synonymous or nonsynonymous
mut_df['class'] = mut_df['Variant_Classification'].apply(lambda x: 'syn' if x in syn else 'nonsyn')

# Step 5: Count N and S per gene
counts = mut_df.groupby(['Hugo_Symbol', 'class']).size().unstack(fill_value=0)
counts = counts.rename(columns={'syn': 'S', 'nonsyn': 'N'})

# Ensure both columns exist
if 'N' not in counts.columns:
    counts['N'] = 0
if 'S' not in counts.columns:
    counts['S'] = 0

# Step 6: Compute naive dN/dS (exclude genes with S = 0)
counts = counts[counts['S'] > 0]
counts['dNdS_naive'] = counts['N'] / counts['S']

# Step 7: View top candidates
top = counts.sort_values('dNdS_naive', ascending=False).head(10)
print(top[['N', 'S', 'dNdS_naive']])


class          N  S  dNdS_naive
Hugo_Symbol                    
TP53         169  2   84.500000
ERBB2         33  1   33.000000
SMAD4         29  1   29.000000
CFH           28  1   28.000000
PIK3CA        82  3   27.333333
CDH9          25  1   25.000000
CNTN4         25  1   25.000000
GRIN3A        24  1   24.000000
THSD1         23  1   23.000000
AMY2B         21  1   21.000000


In [31]:
import warnings
warnings.filterwarnings("ignore", category=ResourceWarning)


In [59]:
pip install mysql-connector-python


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Note: you may need to restart the kernel to use updated packages.


In [60]:
import mysql.connector
import pandas as pd

# Connect to UCSC
conn = mysql.connector.connect(
    host="genome-mysql.soe.ucsc.edu",
    user="genome",
    database="hg19"
)
cursor = conn.cursor()

# Corrected SQL query
query = """
SELECT name2, cdsStart, cdsEnd, exonStarts, exonEnds
FROM refGene;
"""
cursor.execute(query)
rows = cursor.fetchall()

# Create DataFrame
df = pd.DataFrame(rows, columns=["Hugo_Symbol", "cdsStart", "cdsEnd", "exonStarts", "exonEnds"])

# Compute CDS length per transcript
def compute_cds_length(row):
    exon_starts = row['exonStarts']
    exon_ends = row['exonEnds']

    # Decode from bytes to string if needed
    if isinstance(exon_starts, bytes):
        exon_starts = exon_starts.decode("utf-8")
    if isinstance(exon_ends, bytes):
        exon_ends = exon_ends.decode("utf-8")

    # Process comma-separated strings
    starts = list(map(int, exon_starts.strip(',').split(',')))
    ends = list(map(int, exon_ends.strip(',').split(',')))
    cds_start, cds_end = row['cdsStart'], row['cdsEnd']

    length = 0
    for s, e in zip(starts, ends):
        overlap_start = max(s, cds_start)
        overlap_end = min(e, cds_end)
        if overlap_end > overlap_start:
            length += (overlap_end - overlap_start)
    return length


df['cds_length'] = df.apply(compute_cds_length, axis=1)

# Collapse multiple transcripts to max CDS per gene
gene_lengths = (
    df.groupby("Hugo_Symbol")['cds_length']
    .max()
    .reset_index()
)

# Drop 0-length entries
gene_lengths = gene_lengths[gene_lengths['cds_length'] > 0]

# Save
gene_lengths.to_csv("gene_coding_lengths.tsv", sep="\t", index=False)

# Close connection
cursor.close()
conn.close()


In [64]:
# Step 5: Count N and S per gene
counts = (
    mut_df.groupby(['Hugo_Symbol', 'class'])
    .size()
    .unstack(fill_value=0)
    .rename(columns={'syn': 'S', 'nonsyn': 'N'})
    .reset_index()
)

# ✅ Merge mutation counts with gene sizes
df = counts.merge(gene_lengths, on='Hugo_Symbol', how='inner')

# Step 1: Estimate number of sites (opportunities)
df['L_N'] = 0.75 * df['cds_length']
df['L_S'] = 0.25 * df['cds_length']

# Step 2: Compute normalized rates
df['dN'] = df['N'] / df['L_N']
df['dS'] = df['S'] / df['L_S']

# Step 3: Compute size-adjusted dN/dS
df = df[df['dS'] > 0]  # avoid division by zero
df['dNdS_corrected'] = df['dN'] / df['dS']

# Optional: sort for potential drivers
top = df.sort_values('dNdS_corrected', ascending=False).head(10)
print(top[['Hugo_Symbol', 'N', 'S', 'cds_length', 'dNdS_corrected']])


      Hugo_Symbol    N  S  cds_length  dNdS_corrected
14396        TP53  169  2        1182       28.166667
4223        ERBB2   33  1        3885       11.000000
12847       SMAD4   29  1        1659        9.666667
2426          CFH   28  1        3696        9.333333
10207      PIK3CA   82  3        3207        9.111111
2257         CDH9   25  1        2370        8.333333
2773        CNTN4   25  1        3081        8.333333
5570       GRIN3A   24  1        3348        8.000000
13947       THSD1   23  1        2559        7.666667
566         AMY2B   21  1        1536        7.000000


In [65]:
# Step 1: Estimate number of sites (opportunities)
df['L_N'] = 0.75 * df['cds_length']
df['L_S'] = 0.25 * df['cds_length']

# Step 2: Compute normalized rates
df['dN'] = df['N'] / df['L_N']
df['dS'] = df['S'] / df['L_S']

# Step 3: Compute size-adjusted dN/dS
df = df[df['dS'] > 0]  # avoid division by zero
df['dNdS_corrected'] = df['dN'] / df['dS']

# Optional: sort for potential drivers
top = df.sort_values('dNdS_corrected', ascending=False).head(30)
print(top[['Hugo_Symbol', 'N', 'S', 'cds_length', 'dNdS_corrected']])


      Hugo_Symbol    N  S  cds_length  dNdS_corrected
14396        TP53  169  2        1182       28.166667
4223        ERBB2   33  1        3885       11.000000
12847       SMAD4   29  1        1659        9.666667
2426          CFH   28  1        3696        9.333333
10207      PIK3CA   82  3        3207        9.111111
2257         CDH9   25  1        2370        8.333333
2773        CNTN4   25  1        3081        8.333333
5570       GRIN3A   24  1        3348        8.000000
13947       THSD1   23  1        2559        7.666667
566         AMY2B   21  1        1536        7.000000
5827          HGF   19  1        2187        6.333333
15217       VWA5A   19  1        2361        6.333333
3987       EGFLAM   19  1        3054        6.333333
8394         MYH1   19  1        5820        6.333333
9394       OR4C16   19  1         933        6.333333
9366       OR2T12   19  1         963        6.333333
3287        DAAM1   18  1        3237        6.000000
2732        CNGA4   18  1   

In [66]:
# Neutral mutation rate per base (hypothetical; adjust as needed)
mu = 1e-6

# Expected nonsynonymous mutations = mutation rate × number of nonsynonymous sites
df['E_N'] = mu * df['L_N']
from scipy.stats import poisson
from statsmodels.stats.multitest import multipletests

df['pval'] = poisson.sf(df['N'] - 1, df['E_N'])  # P(X ≥ N | λ = E_N)
df['qval'] = multipletests(df['pval'], method='fdr_bh')[1]


In [23]:
!pip install Bio


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 12.9 MB/s eta 0:00:00
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you

In [25]:
!pip install pyfaidx


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [48]:
from pyfaidx import Fasta

# Load genome
fasta = Fasta("hg19.fa")

# Function to fetch context
def get_context(row):
    chrom = row['Chromosome']
    pos = row['Start_Position'] - 1  # 0-based
    try:
        seq = fasta[chrom][pos - 1: pos + 2].seq.upper()  # center base + flanks
        if len(seq) == 3:
            return seq
    except Exception:
        return None
    return None

# Apply to get CONTEXT column
mut_df['CONTEXT'] = mut_df.apply(get_context, axis=1)


In [49]:
print(mut_df['CONTEXT'].dropna().head(5))


Series([], Name: CONTEXT, dtype: float64)


In [44]:
import pandas as pd

# Ensure mutation data is uppercase
mut_df['Reference_Allele'] = mut_df['Reference_Allele'].str.upper()
mut_df['Tumor_Seq_Allele2'] = mut_df['Tumor_Seq_Allele2'].str.upper()
mut_df['CONTEXT'] = mut_df['CONTEXT'].str.upper()

# Define DNA complement map
complement = str.maketrans("ACGT", "TGCA")

# Step 1: Standardize to pyrimidine-centered mutations (i.e., flip G>A to C>T on reverse strand)
def standardize(row):
    ref = row['Reference_Allele']
    alt = row['Tumor_Seq_Allele2']
    context = row['CONTEXT']
    if len(context) != 3 or 'N' in context:
        return pd.Series({'tri_context': None, 'mut_type': None})  # filter bad rows

    if ref in ['G', 'A']:
        # Reverse complement everything
        ref = ref.translate(complement)
        alt = alt.translate(complement)
        context = context.translate(complement)[::-1]

    return pd.Series({'tri_context': context, 'mut_type': f"{ref}>{alt}"})

# Apply the standardization to each row
mut_df[['tri_context', 'mut_type']] = mut_df.apply(standardize, axis=1)

# Drop rows with invalid contexts
mut_df = mut_df.dropna(subset=['tri_context', 'mut_type'])

# Step 2: Count observed mutations for each trinucleotide context and mutation type
spectrum = (
    mut_df.groupby(['tri_context', 'mut_type'])
    .size()
    .unstack(fill_value=0)
)

# Step 3: Normalize per context to get probabilities
trinuc_probs = spectrum.div(spectrum.sum(axis=1), axis=0).to_dict(orient='index')

# Optional sanity check
print("Total contexts:", len(trinuc_probs))
print("Example:", list(trinuc_probs.items())[:3])


Total contexts: 0
Example: []


In [45]:
from Bio.Data import CodonTable
from Bio.Seq import Seq

# Codon → AA mapping
standard_table = CodonTable.unambiguous_dna_by_name["Standard"]
codon_to_aa = {**standard_table.forward_table, **{codon: '*' for codon in standard_table.stop_codons}}

def get_tricontext(seq, i):
    """Return trinucleotide centered at position i"""
    if i == 0 or i == len(seq) - 1:
        return None
    return seq[i-1:i+2]

expected_counts = {}

for gene, cds_seq in gene_cds_dict.items():
    cds_seq = cds_seq.upper()
    E_N, E_S = 0, 0
    
    for i in range(0, len(cds_seq) - 2, 3):  # slide codons
        codon = cds_seq[i:i+3]
        if 'N' in codon or len(codon) < 3:
            continue
        
        ref_aa = codon_to_aa.get(codon, '?')
        
        for j in range(3):  # mutate each base in codon
            ref_nt = codon[j]
            for alt_nt in 'ACGT':
                if alt_nt == ref_nt:
                    continue
                
                # Simulate mutation
                mut_codon = list(codon)
                mut_codon[j] = alt_nt
                mut_codon = ''.join(mut_codon)
                alt_aa = codon_to_aa.get(mut_codon, '?')

                # Get context for mutation
                pos_in_seq = i + j
                context = get_tricontext(cds_seq, pos_in_seq)
                if context is None or 'N' in context:
                    continue
                
                # Mutation type (pyrimidine-centered)
                mut_type = f"{ref_nt}>{alt_nt}"
                if ref_nt in 'AG':
                    # flip everything for pyrimidine standardization
                    complement = str.maketrans('ACGT', 'TGCA')
                    context = context.translate(complement)[::-1]
                    mut_type = f"{ref_nt.translate(complement)}>{alt_nt.translate(complement)}"

                # Get probability from trinuc spectrum
                prob = trinuc_probs.get(context, {}).get(mut_type, 0)
                
                # Skip if mutation is invalid or missing
                if ref_aa == '?' or alt_aa == '?':
                    continue
                
                # Add to appropriate bucket
                if alt_aa == ref_aa:
                    E_S += prob
                else:
                    E_N += prob

    expected_counts[gene] = {'E_N': E_N, 'E_S': E_S}


KeyboardInterrupt: 

In [67]:

expected_df = pd.DataFrame.from_dict(expected_counts, orient='index').reset_index()
expected_df = expected_df.rename(columns={'index': 'Hugo_Symbol'})
merged = obs_df.merge(expected_df, on='Hugo_Symbol', how='inner')



In [68]:
from scipy.stats import poisson
from statsmodels.stats.multitest import multipletests

merged['pval'] = poisson.sf(merged['N'] - 1, merged['E_N'])
merged['qval'] = multipletests(merged['pval'], method='fdr_bh')[1]
top_drivers = merged.sort_values('pval').reset_index(drop=True)
print(top_drivers[['Hugo_Symbol', 'N', 'E_N', 'pval', 'qval']].head(10))



  Hugo_Symbol   N  E_N  pval  qval
0        A1BG   4    0   0.0   0.0
1        AMD1   6    0   0.0   0.0
2      AMDHD2   4    0   0.0   0.0
3       AMELX   2    0   0.0   0.0
4       AMELY   2    0   0.0   0.0
5       AMER1  11    0   0.0   0.0
6       AMER2  14    0   0.0   0.0
7       AMER3   5    0   0.0   0.0
8        AMFR   7    0   0.0   0.0
9         AMH   2    0   0.0   0.0


In [47]:
print(mut_df['CONTEXT'].dropna().apply(len).value_counts())


Series([], Name: CONTEXT, dtype: int64)
